In [19]:
import pandas as pd
import time
import numpy as np
import struct
import matplotlib.pyplot as plt
from cr_data_inspection_functions import *
from lwa_antpos import mapping
from scipy.optimize import curve_fit
import math
import scipy.stats as st

## load array map
array_map=pd.read_csv('array-map-5-22-2023.csv')
xdict={}
ydict={}
zdict={}
for i,n in enumerate(array_map['antname']):
    xdict[n]=array_map['x'][i]
    ydict[n]=array_map['y'][i]
    zdict[n]=array_map['elevation'][i]
arraymapdictionaries=[xdict,ydict,zdict]

In [24]:
#saved_events_indices=np.load('/data0/cosmic-ray-data/2023May3/overnight1683180640.2699194.indices_first_cuts.npy')
saved_events_indices=np.load('/data0/cosmic-ray-data/2023May3-dataproducts/early_attempts/2023May3-dataproductsovernight1683180803.759128.indices_cuts2.npy')

fname='/data0/cosmic-ray-data/2023May3/overnight1683180640.2699194.dat'




reloaded_events=[]
for i in saved_events_indices:
    event_records=parsefile(fname,start_ind=i,end_ind=704 ) #note that start_ind counts from the beginning of the file and end_ind counts from start_ind 
    reloaded_events.append(event_records)
print(len(reloaded_events))

3


In [ ]:
print(len(event_records))
merged=mergepolarizations(event_records,arraymapdictionaries)
for i,m in enumerate(merged):
    if 'rmsB' in m.keys():
        pass
    else:
        print(i)
    

In [41]:
for event in [reloaded_events[0]]:  
    mergedrecords=mergepolarizations(event,arraymapdictionaries)

    rmsA=np.asarray([record['rmsA'] for record in mergedrecords])
    peakA=np.asarray([record['peakA'] for record in mergedrecords])
    index_peak_A=np.asarray([record['index_peak_A'] for record in mergedrecords])

    rmsB=np.asarray([record['rmsB'] for record in mergedrecords])
    peakB=np.asarray([record['peakB'] for record in mergedrecords])
    index_peak_B=np.asarray([record['index_peak_B'] for record in mergedrecords])
    peak_to_rmsA=peakA/rmsA
    peak_to_rmsB=peakB/rmsB

    xcoords=np.asarray([record['x'] for record in mergedrecords])
    ycoords=np.asarray([record['y'] for record in mergedrecords])
    zcoords=np.asarray([record['z'] for record in mergedrecords])
    timestamps=np.asarray([record['timestamp'] for record in mergedrecords])
    min_time=np.min(timestamps)

    t_rel_A=index_peak_A + timestamps - min_time
    t_rel_B=index_peak_B + timestamps - min_time

    #make antenna cut based on rms
    maximum_ok_rms=45
    minimum_ok_rms=25
    cutA = np.logical_and(rmsA >minimum_ok_rms, rmsA <maximum_ok_rms)
    cutB = np.logical_and(rmsB >minimum_ok_rms, rmsB <maximum_ok_rms)
    
    cmin=0
    cmax=11
    minsnr=6
    minstrongdetections=50
    plot_peak_to_rms_ratio(peak_to_rmsA,cutA,peak_to_rmsB,cutB,xcoords,ycoords,cmin,cmax)
    strongAdetections=np.sum((peak_to_rmsA[cutA])>minsnr)
    strongBdetections=np.sum((peak_to_rmsB[cutB])>minsnr)
    print(strongAdetections,strongBdetections,strongAdetections/strongBdetections)
    strongestAsnr=np.max(peak_to_rmsA[cutA])
    strongestBsnr=np.max(peak_to_rmsB[cutB])
    
    #plot timeseries (both polarizations) for highest snr antenna of each polarization
    for r, record in enumerate(mergedrecords):
        if record['peakA']/record['rmsA'] ==strongestAsnr:
            kurtosisA=st.kurtosis(record['polA_data'])
            kurtosisB=st.kurtosis(record['polB_data'])
            plt.figure(figsize=(20,4))
            plt.suptitle(record['antname']+ ' x = '+str(record['x'])+' y = '+str(record['y'])+ ' rms A = '+str(record['rmsA'])+ 'kurtosis A, B = '+str(kurtosisA)+', '+str(kurtosisB))
            plt.subplot(121)
            plt.plot(record['polA_data'])
            plt.xlabel('Position in buffer for '+ record['antname'] +'Polarization A')
            plt.ylabel('Voltage [ADC units]')
            plt.subplot(122)
            plt.plot(record['polB_data'])
            plt.xlabel('Position in buffer for '+ record['antname'] +'Polarization B')
            plt.ylabel('Voltage [ADC units]')

            plt.figure(figsize=(20,4))
            plt.subplot(121)
            #note the different zoom for each pol
            plt.plot(record['polA_data'])
            plt.xlim(record['index_peak_A']-50,record['index_peak_A']+150)
            plt.xlabel('Position in buffer for '+ record['antname'] +'Polarization A')
            plt.ylabel('Voltage [ADC units]')
            plt.subplot(122)
            plt.plot(record['polB_data']) 
            plt.xlim(record['index_peak_B']-50,record['index_peak_B']+150)
            plt.xlabel('Position in buffer for '+ record['antname'] +'Polarization B')
            plt.ylabel('Voltage [ADC units]')
            plt.show()

        if record['peakB']/record['rmsB'] ==strongestBsnr: 
            kurtosisA=st.kurtosis(record['polA_data'][:2000])
            kurtosisB=st.kurtosis(record['polB_data'][:2000])

            plt.figure(figsize=(20,4))
            plt.suptitle(record['antname']+ ' x = '+str(record['x'])+' y = '+str(record['y'])+ ' rms B = '+str(record['rmsB'])+ 'kurtosis A, B = '+str(kurtosisA)+', '+str(kurtosisB))
            plt.subplot(121)
            plt.plot(record['polA_data'])
            plt.xlabel('Position in buffer for '+ record['antname'] +'Polarization A')
            plt.ylabel('Voltage [ADC units]')
            plt.subplot(122)
            plt.plot(record['polB_data'])
            plt.xlabel('Position in buffer for '+ record['antname'] +'Polarization B')
            plt.ylabel('Voltage [ADC units]')

            plt.figure(figsize=(20,4))
            plt.subplot(121)
            #note the different zoom for each pol
            plt.plot(record['polA_data'])
            plt.xlim(record['index_peak_A']-50,record['index_peak_A']+150)
            plt.xlabel('Position in buffer for '+ record['antname'] +'Polarization A')
            plt.ylabel('Voltage [ADC units]')
            plt.subplot(122)
            plt.plot(record['polB_data']) 
            plt.xlim(record['index_peak_B']-50,record['index_peak_B']+150)
            plt.xlabel('Position in buffer for '+ record['antname'] +'Polarization B')
            plt.ylabel('Voltage [ADC units]')
        plt.show()

KeyError: 'rmsB'

In [42]:
print(mergedrecords[0].keys())

dict_keys(['antname', 'x', 'y', 'z', 'timestamp', 'board_id', 'this_board_triggered', 'coincidence_threshold', 'veto_threshold', 'trigger_power_threshold', 'veto_power_threshold', 'coincidence_window', 'veto_coincidence_window', 'trigger_role_A', 'antenna_id_A', 'veto_role_A', 'polA_data', 'rmsA', 'index_peak_A', 'peakA', 'trigger_role_B', 'antenna_id_B', 'veto_role_B', 'polB_data', 'rmsB', 'index_peak_B', 'peakB'])


In [43]:
record.keys()

dict_keys(['antname', 'x', 'y', 'z', 'timestamp', 'board_id', 'this_board_triggered', 'coincidence_threshold', 'veto_threshold', 'trigger_power_threshold', 'veto_power_threshold', 'coincidence_window', 'veto_coincidence_window', 'trigger_role_A', 'antenna_id_A', 'veto_role_A', 'polA_data', 'rmsA', 'index_peak_A', 'peakA'])

In [44]:
for i,record in enumerate(mergedrecords):
    rmsB=record['rmsB']


KeyError: 'rmsB'

In [45]:
print(i)

349
